# Set Custom Loss


In [103]:
# Create dataset and model
import sys
sys.path.append('/home/silvia/Documents/GitHub/primoprogetto/Codici/')  # PER importare cose che non sono nella stessa directory
from Classe_sismogramma_v3 import ClasseDataset
import keras 
import numpy as np
import tensorflow as tf

pat_tent = '/home/silvia/Documents/GitHub/primoprogetto/Codici/Tentativi/'
tentativo = 52
model = keras.models.load_model(pat_tent+str(tentativo)+'/Tentativo_'+str(tentativo)+'.hdf5')

hdf5pol = '/home/silvia/Desktop/Data/Instance_Data/Tre_4s/data_Velocimeter_4s_Normalizzate_New1-1.hdf5'
csvpol = '/home/silvia/Desktop/Data/Instance_Data/Tre_4s/metadata_Velocimeter_4s_Normalizzate_New1-1.csv'

hdf5und = "/home/silvia/Desktop/Data/Instance_Data/Undecidable/Instance_undecidable_data_tot_no0.hdf5"
csvund = "/home/silvia/Desktop/Data/Instance_Data/Undecidable/Instance_undecidable_metadata_tot_no0.csv"

Dpol, Dund = (ClasseDataset(),ClasseDataset())
Dpol.leggi_custom_dataset(hdf5pol,csvpol)
Dund.leggi_custom_dataset(hdf5und,csvund)
semi_amp = 80
lung = len(Dund.sismogramma[0])

N_pol = len(Dpol.sismogramma)
xtest_pol = np.zeros((N_pol, semi_amp * 2))
for k in range(N_pol):
    xtest_pol[k] = Dpol.sismogramma[k][lung // 2 - semi_amp : lung // 2 + semi_amp]
y_test_true_pol = np.array([Dpol.metadata["trace_polarity"][_] == "positive" for _ in range(N_pol)])
y_test_true_pol = y_test_true_pol + 0

N_und = len(Dund.sismogramma)
xtest_und = np.zeros((N_und, semi_amp * 2))
for k in range(N_und):
    xtest_und[k] = Dund.sismogramma[k][lung // 2 - semi_amp : lung // 2 + semi_amp]
y_test_true_und = np.array([0.5 for _ in range(N_und)])


In [37]:
# ATTENTO A MONITOR di EarlyStopping!!!!!!!!!!! (voglio monitorare solo la cosa di prima???)
from keras.utils.metrics_utils import binary_matches 
from keras import backend
import keras

def calc_CE(y_true,y_pred):
    """Compute mean crossentropy on data where y_true == 0 or y_true == 1""" # (Verified, it works)
    # CE_bool shows where terms that contribute to crossentropy are 
    CE_bool = tf.cast(tf.equal(y_true, 1), backend.floatx()) + tf.cast(tf.equal(y_true, 0), backend.floatx())
    CE = tf.keras.losses.binary_crossentropy(CE_bool*tf.cast(y_true, backend.floatx()), CE_bool*tf.cast(y_pred, backend.floatx())) * len(CE_bool)/tf.reduce_sum(CE_bool)
    return CE


def calc_FLT1(y_true,y_pred):
    """Compute MSE on data where y_true == 0.5""" # (Verified, it works)
    # FLT_bool shows where terms that contribute to FLT are  
    FLT_bool = tf.cast(tf.equal(y_true, 0.5), backend.floatx())
    FLT = tf.keras.losses.mean_squared_error(FLT_bool*tf.cast(y_true, backend.floatx()), FLT_bool*tf.cast(y_pred, backend.floatx())) * len(FLT_bool)/tf.reduce_sum(FLT_bool)
    return FLT

def calc_FLT2(y_true,y_pred):
    """Compute MSE on data where y_true == 0.5""" # (Verified, it works)
    # FLT_bool shows where terms that contribute to FLT are  
    FLT_bool = tf.cast(tf.equal(y_true, 0.5), backend.floatx())
    #FLT = tf.maximum(FLT_bool*tf.cast(y_true, backend.floatx()), FLT_bool*tf.cast(y_pred, backend.floatx())) * len(FLT_bool)/tf.reduce_sum(FLT_bool)
    return 0 #FLT

def my_loss(lamb):
    """
    This custom loss for i-th sample is computed as follows:
    if yti == 0 or yti == 1
        loss = binarycrossentropy(yti, ypi)
    if yti == 0.5:
        loss = lamb * (yti - ypi)**2

    I chose to represent waveforms with no definite polarity (or no events or other) labeled with yt=0.5.
    I instruct the network to provide a response of 0.5 for out-of-distribution waveforms
    """
    def calc_loss(y_true,y_pred):
        y_true = tf.convert_to_tensor(y_true)
        y_pred = tf.convert_to_tensor(y_pred)
        
        # CE CROSS ENTROPY term (verified it works)
        # FLT "ENFORCING LOW CONFIDENCE" term. On out of distribution data
        # Verified, it works
        return (calc_CE(y_true,y_pred) + lamb * calc_FLT1(y_true,y_pred)) 
    return calc_loss


def calc_lamb(model, y_true, xtrain):
    """
    Calculating the lamb parameter 
    """
    y_pred = model.predict(xtrain)

    return calc_CE(y_true, y_pred) / calc_FLT1(y_true, y_pred)

def my_acc(y_true,y_pred):
    """
    Compute accuracy only on definite polarity waveforms, exclude from the count other!
    binary_matches return a tf.tensor, whoose value in i-th pos is 1 
    when (yp_i > 0.5 and yt_i==1) or ( yp_i < 0.5 and yt_i==0), 
    else 0 (0.5 is the threshold, can be changed)
    """
    # return numb of matches / (numb yt==0 + numb yt==1)
    return tf.math.reduce_sum(binary_matches(y_true, y_pred)) / tf.math.reduce_sum (tf.cast(tf.equal(y_true, 1), backend.floatx()) + tf.cast(tf.equal(y_true, 0), backend.floatx()))


class Cambia_Loss_Epoca(keras.callbacks.Callback):
    "The model is an attribute of tf.keras.callbacks.Callback, so you can access it directly with self.model"
    def __init__(self, xtrain, ytrain_true):
        self.validation_data = None
        self.model = None
        # Whether this Callback should only run on the chief worker in a
        # Multi-Worker setting.
        # TODO(omalleyt): Make this attr public once solution is stable.
        self._chief_worker_only = None
        self._supports_tf_logs = False
        self.xtrain = xtrain
        self.y_true = ytrain_true
    
    def on_epoch_end(self):
        """Compute mean loss on definite polarity, separated from loss on out-of-distribution and set the lambda term in the way that their means match??"""
        y_pred = self.model.predict(self.xtrain)
        
        lamb = calc_lamb(self.model, self.y_true, self.xtrain)
        # if FLT > 2*CE or CE > 2*FLT:
        if lamb > 2 or lamb < 0.5: 
            self.model.compile(
                                # optimizer=optimizers.SGD(momentum=momento),  # TODO CAMBIA
                                loss=my_loss(lamb),
                                metrics=[my_acc]
                            )
            print("Ho cambiato, lamb=", lamb)



    
# model.compile(
#     # optimizer=optimizers.SGD(momentum=momento),  # TODO CAMBIA
#     loss=my_loss(calc_mean_loss_for_allineare_mean(8)),
#     metrics=[my_acc]
# )


# storia = model.fit (bla
#                     bla
#                     bla
#                     callbacks=[EarlyStopping(patience=pazienza,  restore_best_weights=True, monitor=my_starting_loss),
#                                Cambia_Loss_Epoca()]
#                     )

# Checking things

In [46]:
# verify Cross entropy and my method are same 
from keras import backend
import tensorflow as tf
backend.floatx()
ytrue = [0 for i in range(11)] + [0.5 for i in range(11)]  + [1 for i in range(11)] 
ypred = [i/33.0 for i in range(33)]
CE_bool = tf.cast(tf.equal(ytrue, 1), backend.floatx()) + tf.cast(tf.equal(ytrue, 0), backend.floatx())

# TERMINE DI CROSS ENTROPY VERO
a1 = tf.keras.losses.binary_crossentropy(ytrue[0:11] + ytrue[22:],ypred[0:11] + ypred[22:])

# TERMINE DI CROSS ENTROPY "Mio"
#a2 = tf.keras.losses.binary_crossentropy(CE_bool*tf.cast(ytrue, backend.floatx()), CE_bool*tf.cast(ypred, backend.floatx())) * len(CE_bool)/tf.reduce_sum(CE_bool)
a2 = calc_CE(ytrue, ypred)
#lossi = my_loss(0.0)
#a2 = lossi(ytrue, ypred)
float(a1-a2)

-1.4901161193847656e-08

In [13]:
# verify MSE and my method are same  
from keras import backend
import tensorflow as tf
backend.floatx()
ytrue = [0 for i in range(11)] + [0.5 for i in range(11)]  + [1 for i in range(11)] 
ypred = [i/33.0 for i in range(33)]
FLT_bool = tf.cast(tf.equal(ytrue, 0.5), backend.floatx())

# TERMINE DI MSE VERO
a1 = tf.keras.losses.mean_squared_error(ytrue[11:22] ,ypred[11:22])

# TERMINE DI MSE "Mio"
#a2 = tf.keras.losses.mean_squared_error(FLT_bool*tf.cast(ytrue, backend.floatx()), FLT_bool*tf.cast(ypred, backend.floatx())) * len(FLT_bool)/tf.reduce_sum(FLT_bool)
#a2 = calc_FLT(ytrue, ypred)
lossi = my_loss(1.0)
a2 = lossi(ytrue, ypred)
print(a1)
print(a2)
print(a1-a2)

tf.Tensor(0.009412304, shape=(), dtype=float32)
tf.Tensor(0.009412304, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)


In [95]:
# Verufy model.loss is binary_crossentropy
tf.keras.losses.binary_crossentropy is model.loss # output is True

True

In [2]:
import tensorflow as tf

# Definisci due tensori con la stessa dimensione
tensor1 = tf.constant([1, 30, 5, 7])
tensor2 = tf.constant([2, 4, 6, 8])

# Calcola il massimo elemento per elemento
max_tensor = tf.maximum(tensor1, tensor2)

# Stampa il risultato
print(max_tensor.numpy())

[ 2 30  6  8]


# verify 0-entries in calc_CE does not affect gradients

In [1]:

from keras import backend
import tensorflow as tf
from keras.layers import Dense, Conv1D, Flatten
import numpy as np
def calc_CE(y_true,y_pred):
    """Compute mean crossentropy on data where y_true == 0 or y_true == 1""" # (Verified, it works)
    # CE_bool shows where terms that contribute to crossentropy are 
    CE_bool = tf.cast(tf.equal(y_true, 1), backend.floatx()) + tf.cast(tf.equal(y_true, 0), backend.floatx())
    CE = tf.keras.losses.binary_crossentropy(CE_bool*tf.cast(y_true, backend.floatx()), CE_bool*tf.cast(y_pred, backend.floatx())) * len(CE_bool)/tf.reduce_sum(CE_bool)
    #CE = tf.keras.losses.binary_crossentropy(tf.cast(y_true, backend.floatx()), tf.cast(y_pred, backend.floatx()))
    return CE

def create_identity(x):
    vinput_shape = (None, 20, 1)
    identity = Conv1D(filters=1, kernel_size=1, activation=None, input_shape=vinput_shape[1:])
    x_0 = x
    x_0 = np.expand_dims(x_0, axis=0)
    x_0 = np.expand_dims(x_0, axis=-1)
    aia = identity(x_0)                
    identity.set_weights([np.array([[[1.0]]]), np.array([0.0])])
    return identity


ytrue = [0 for i in range(1)] + [0.5 for i in range(1)]  + [1 for i in range(1)] 
x = np.array([[i/20.0 +j for i in range(20)]for j in range(3)])



ID = create_identity(x[0])
fl = Flatten()
layD = Dense(1, activation="sigmoid")

2023-10-16 10:42:32.650581: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-16 10:42:34.463373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 10:42:35.057609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5225 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:9e:00.0, compute capability: 7.5


In [5]:
# verifica se faccio binary crossentropy va bene lo stesso. poi verifica manualemnte i gradienti??
with tf.GradientTape() as tape:
    indx = 2
    x_0 = x[indx]
    x_0 = np.expand_dims(x_0, axis=0)
    x_0 = np.expand_dims(x_0, axis=-1)
    act = ID(x_0)
    act2 = fl(act)
    act3 = layD(act2)
    Loss = calc_CE([ytrue[indx]],[act3])
    grad = tape.gradient(Loss,act)
grad = np.array(grad)
grad = grad[0]
print("grad ",grad)
print("yrue ",ytrue[indx])

grad  [[ 0.01531619]
 [-0.00386189]
 [-0.00439651]
 [-0.02264871]
 [ 0.01406922]
 [-0.00192071]
 [ 0.01530548]
 [ 0.00281429]
 [-0.01425789]
 [-0.00090934]
 [-0.02470191]
 [ 0.00739589]
 [-0.01277691]
 [-0.00793162]
 [-0.02568799]
 [ 0.00436738]
 [-0.00963266]
 [-0.00122552]
 [ 0.02891251]
 [-0.0202882 ]]
yrue  1
